In [1]:
!pip install tpot

     |████████████████████████████████| 87 kB 3.1 MB/s 
     |████████████████████████████████| 160 kB 45.3 MB/s 
     |████████████████████████████████| 173.6 MB 7.3 kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11956 sha256=9f503d523bbc67e64019088788ef2ed92a07b4cacfd5eddf2098c6023a8959f8
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tpot import TPOTClassifier

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/mgovindasamy/tpot-tutorial/master/Data.csv")

In [7]:
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,1,0,5849,0.0,126,360.0,1,2,1
1,1,1,1,1,0,4583,1508.0,128,360.0,1,0,0
2,1,1,0,1,1,3000,0.0,66,360.0,1,2,1
3,1,1,0,0,0,2583,2358.0,120,360.0,1,2,1
4,1,0,0,1,0,6000,0.0,141,360.0,1,2,1


In [8]:
target_col = "Loan_Status"
X = df.loc[:, df.columns != target_col]
y = df.loc[:, target_col]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=2021)

In [10]:
tpot = TPOTClassifier(generations=5, 
                      population_size=50, 
                      verbosity=2, 
                      n_jobs=-1)

In [11]:
tpot.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7820449391877963

Generation 2 - Current best internal CV score: 0.7820449391877963

Generation 3 - Current best internal CV score: 0.7820449391877963

Generation 4 - Current best internal CV score: 0.7820449391877963

Generation 5 - Current best internal CV score: 0.7820449391877963

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=False, criterion=gini, max_features=0.6500000000000001, min_samples_leaf=12, min_samples_split=16, n_estimators=100)


TPOTClassifier(generations=5, n_jobs=-1, population_size=50, verbosity=2)

In [12]:
tpot.score(X_test, y_test)

0.7235772357723578

In [13]:
tpot.export('tpot_pipeline.py')

In [15]:
!cat tpot_pipeline.py

import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=None)

# Average CV score on the training set was: 0.7820449391877963
exported_pipeline = ExtraTreesClassifier(bootstrap=False, criterion="gini", max_features=0.6500000000000001, min_samples_leaf=12, min_samples_split=16, n_estimators=100)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


In [16]:
tpot.evaluated_individuals_

{'BernoulliNB(ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier__criterion=entropy, ExtraTreesClassifier__max_features=0.45, ExtraTreesClassifier__min_samples_leaf=9, ExtraTreesClassifier__min_samples_split=17, ExtraTreesClassifier__n_estimators=100), BernoulliNB__alpha=0.001, BernoulliNB__fit_prior=True)': {'crossover_count': 0,
  'generation': 5,
  'internal_cv_score': 0.7820449391877963,
  'mutation_count': 2,
  'operator_count': 2,
  'predecessor': ('BernoulliNB(input_matrix, BernoulliNB__alpha=0.001, BernoulliNB__fit_prior=True)',)},
 'BernoulliNB(Nystroem(input_matrix, Nystroem__gamma=0.45, Nystroem__kernel=chi2, Nystroem__n_components=6), BernoulliNB__alpha=0.1, BernoulliNB__fit_prior=True)': {'crossover_count': 0,
  'generation': 1,
  'internal_cv_score': 0.6965367965367965,
  'mutation_count': 1,
  'operator_count': 2,
  'predecessor': ('BernoulliNB(Nystroem(input_matrix, Nystroem__gamma=0.45, Nystroem__kernel=linear, Nystroem__n_co

In [17]:
tpot.fitted_pipeline_

Pipeline(steps=[('extratreesclassifier',
                 ExtraTreesClassifier(max_features=0.6500000000000001,
                                      min_samples_leaf=12,
                                      min_samples_split=16))])